In [26]:
import sys
import os
import pandas as pd

import socket
import cryptography


sys.path.append('/app/EZPZ_Project') #db 연동정보 경로
sys.path.append('/app/EZPZ_Project/modules/crawlers/job_post') # 채용공고 경로

from privates.ezpz_db import *

In [27]:
import wanted_recruit_crawler

In [145]:
conn = get_connection()
cur = conn.cursor()

In [ ]:
cur.execute('delete from recruit_info')
cur.execute('delete from comp_news')

In [34]:
comp_list=['삼성전자(주)','(주)카카오','네이버(주)']

In [35]:
cur.execute('desc recruit_info') #여기가 있어야지 news_crawl이 됨
for i in cur:
    print(i)

('recruit_uid', 'bigint', 'NO', 'PRI', None, 'auto_increment')
('comp_uid', 'bigint', 'NO', 'MUL', None, '')
('recruit_url', 'varchar(2000)', 'NO', '', None, '')
('recruit_position', 'varchar(100)', 'NO', '', None, '')
('recruit_thumb', 'varchar(2000)', 'YES', '', None, '')
('create_date', 'char(8)', 'NO', '', None, '')
('modify_date', 'char(8)', 'NO', '', None, '')


In [36]:
cur.execute('select * from recruit_info order by comp_uid') #여기가 있어야지 news_crawl이 됨
for i in cur:
    print(i)

(149197, 85, 'https://www.wanted.co.kr/wd/149197', 'Inference Engine Engineer', 'https://static.wanted.co.kr/images/company/22504/u2ryao0fwotejf3h__1080_790.jpg', '00000000', '00000000')
(151828, 85, 'https://www.wanted.co.kr/wd/151828', 'Applied AI/ML Engineer', 'https://static.wanted.co.kr/images/company/22504/u2ryao0fwotejf3h__1080_790.jpg', '00000000', '00000000')
(156751, 85, 'https://www.wanted.co.kr/wd/156751', 'Biology AI Researcher', 'https://static.wanted.co.kr/images/company/22504/u2ryao0fwotejf3h__1080_790.jpg', '00000000', '00000000')
(159350, 85, 'https://www.wanted.co.kr/wd/159350', 'Backend Engineer(AI Image Service)', 'https://static.wanted.co.kr/images/company/22504/u2ryao0fwotejf3h__1080_790.jpg', '00000000', '00000000')
(171904, 85, 'https://www.wanted.co.kr/wd/171904', '[카카오헤어샵] 백엔드 개발자 (미들-시니어)', 'https://static.wanted.co.kr/images/company/2671/txrozr2bl2gyymzy__1080_790.jpg', '00000000', '00000000')
(172569, 85, 'https://www.wanted.co.kr/wd/172569', 'iOS개발자', 'ht

In [ ]:
"""1.회사리스트를 받는다
2.채용공고에서 (주)를 뺀다
3.채용공고 크롤링해서 df받아옴
4.df에서 uid만 리스트로 따로 꺼내옴
크롤링하면 채용공고 5개를 받겠지 (카카오 다 다른 채용공고 uid겠지)
원래 테이블엔 6개의 채용공고가 있었어( 다 다른 uid)
그러면 comp_uid를 6개를 받아서
5.테이블에서 comp_uid를 빼와서 6개짜리 리스트로만들고
len비교
if len comp_uid 테이블 길이> 크롤링 df 길이: #없어진 채용공고 있는거니까
	테이블 uid_list - 크롤링 df uid 차집합리스트 애들을 
	차집합 리스트로 for 문돌려서 하나하나 들어가있는지 확인해서 삭제
elif len comp_uid 테이블 길이 = 크롤링 df길이:
	pass하는식으로 넘기고
else #크롤링 df가 더 긴경우
	크롤링 df uid -테이블 uid리스트 차집합 리스트 애들을
	차집합 리스트로 for문돌려서 하나하나 채용공고 
	insert where uid= {new_recruit}"""

In [ ]:
uid_table_list=[]
cur.execute("SELECT recruit_info.recruit_uid FROM recruit_info JOIN comp_info ON recruit_info.comp_uid = comp_info.comp_uid WHERE comp_info.comp_name = %s", (comp))


In [58]:
cur.execute("SELECT recruit_info.recruit_uid FROM recruit_info JOIN comp_info ON recruit_info.comp_uid = comp_info.comp_uid WHERE comp_info.comp_name = %s", '(주)카카오')
uid_table_list=[]
for r_uid in cur:
    uid_table_list.append(r_uid[0])

In [69]:
cur.execute('desc recruit_info')
for i in cur:
    print(i)

('recruit_uid', 'bigint', 'NO', 'PRI', None, 'auto_increment')
('comp_uid', 'bigint', 'NO', 'MUL', None, '')
('recruit_url', 'varchar(2000)', 'NO', '', None, '')
('recruit_position', 'varchar(100)', 'NO', '', None, '')
('recruit_thumb', 'varchar(2000)', 'YES', '', None, '')
('create_date', 'char(8)', 'NO', '', None, '')
('modify_date', 'char(8)', 'NO', '', None, '')


In [139]:
#테스트용으로 아무거나 delete =>공고 추가되나 확인용
cur.execute('delete from recruit_info where recruit_position = "[신입] iOS개발"')

1

In [ ]:
#테스트용으로 아무거나 insert =>공고 삭제되나 확인용
sql = 'insert into recruit_info '
sql += '    (comp_uid, recruit_uid, recruit_url, recruit_position, recruit_thumb, create_date, modify_date) '
sql += 'values ( '
sql += f'   "86", "3955", "https://www.wanted.co.kr/wd/3955", "나는 김형선", "https://static.wanted.co.kr/images/company/3342/nmshwilloxvcqlxq__1080_790.png" '
sql += f'    , "{"00000000"}", "{"00000000"}" '
sql += ') '
cur.execute(sql)
cur.execute('')

In [142]:
#추가 삭제는 요걸로 확인
cur.execute('select * from recruit_info')
for i in cur:
    print(i)

(39622, 86, 'https://www.wanted.co.kr/wd/39622', '카카오/네이버 연계 시스템 서버 개발자', 'https://static.wanted.co.kr/images/company/3342/nmshwilloxvcqlxq__1080_790.png', '00000000', '00000000')
(149197, 85, 'https://www.wanted.co.kr/wd/149197', 'Inference Engine Engineer', 'https://static.wanted.co.kr/images/company/22504/u2ryao0fwotejf3h__1080_790.jpg', '00000000', '00000000')
(151828, 85, 'https://www.wanted.co.kr/wd/151828', 'Applied AI/ML Engineer', 'https://static.wanted.co.kr/images/company/22504/u2ryao0fwotejf3h__1080_790.jpg', '00000000', '00000000')
(156751, 85, 'https://www.wanted.co.kr/wd/156751', 'Biology AI Researcher', 'https://static.wanted.co.kr/images/company/22504/u2ryao0fwotejf3h__1080_790.jpg', '00000000', '00000000')
(159350, 85, 'https://www.wanted.co.kr/wd/159350', 'Backend Engineer(AI Image Service)', 'https://static.wanted.co.kr/images/company/22504/u2ryao0fwotejf3h__1080_790.jpg', '00000000', '00000000')
(169809, 86, 'https://www.wanted.co.kr/wd/169809', '네트워크 엔지니어 (춘천 네이버I

In [144]:
#함수테스트완료
recruit_info_update(comp_list)

삼성전자
삼성전자(주) 변동없음
카카오
(주)카카오 변동없음
네이버
네이버(주) 변동없음


In [143]:
def recruit_info_update(comp_list):
    for comp in comp_list:
        #채용공고는 주 붙어있으면 안됨 제거 전처리
        #2
        recruit_comp = comp.replace('(주)',"")
        print(recruit_comp)

        #3
        recruit_info_df=wanted_recruit_crawler.get_recruit_info(recruit_comp, csv_save=False) # 원티드 기업정보 크롤러 모듈

        #3.5 uid 집합으로 돼있어서 int로 변환
        new_i=[] #집합인 uid를 int로 바꿔준 값을 넣어준 리스트/ 테이블에 넣기위한 전처리 (비교하려면 먼저 전처리부터)
        for i in range(len(recruit_info_df['recruit_uid'])):
            new_i.append(list(recruit_info_df['recruit_uid'][i])[0])
        recruit_info_df['recruit_uid']=new_i #int 값으로 컬럼 대체 # 데이터프레임 있는상태 여기서 다 돌려야됨


        #기업별(for 문 한번당 기업하나) recruit_uid를 넣어준 리스트
        #회사이름이 comp_info와 같은 recruit_info에서 recruit_info_uid 정보 전부 빼옴
        cur.execute("SELECT recruit_info.recruit_uid FROM recruit_info JOIN comp_info ON recruit_info.comp_uid = comp_info.comp_uid WHERE comp_info.comp_name = %s", (comp))
        uid_table_list=[]

        for r_uid in cur:
            uid_table_list.append(r_uid[0])

        if len(recruit_info_df['recruit_uid']) > len(uid_table_list): #추가해야될 채용공고가 있는경우: insert
            #차집합으로 크롤링데이터엔 있고 테이블엔 없는 것들 찾아서 추가
            sub_set = [x for x in recruit_info_df['recruit_uid'].tolist() if x not in uid_table_list]
            print(recruit_info_df['recruit_uid'].tolist())
            print(uid_table_list)
            print(sub_set)
            #차집합만을 포함한 데이터프레임 만들기(insert목록)
            filtered_recruit_info_df = recruit_info_df[recruit_info_df['recruit_uid'].isin(sub_set)]
            #채용공고에 넣어줄 comp_id
            cur.execute(f'select comp_uid from comp_info where comp_name = "{comp}"')
            comp_uid=cur.fetchall()[0][0]
            print(comp_uid)

            for index, row in filtered_recruit_info_df.iterrows():
                sql = 'insert into recruit_info '
                sql += '    (comp_uid, recruit_uid, recruit_url, recruit_position, recruit_thumb, create_date, modify_date) '
                sql += 'values ( '
                sql += f'   "{comp_uid}", "{row["recruit_uid"]}", "{row["recruit_url"]}", "{row["recruit_position"]}", "{row["recruit_thumb"]}" '
                sql += f'    , "{"00000000"}", "{"00000000"}" '
                sql += ') '
                cur.execute(sql)
                #for i in cur:
                #    print(i)
            print(f'{comp} 공고 추가완료')
            #print(filtered_recruit_info_df)
        elif len(recruit_info_df['recruit_uid']) == len(uid_table_list) and sorted(recruit_info_df['recruit_uid']) == sorted(uid_table_list) : # 아예 똑같음 채용공고 변동사항 없음
            print(f'{comp} 변동없음')
            continue #채용공고 변동없이 다음기업 찾기로 넘어감

        else: #삭제해야될 채용공고가 있는경우: delete
            #차집합으로 테이블엔 있고 크롤링데이터엔 없는 것들 찾아서 삭제 delete해줄 애들
            sub_set = [x for x in  uid_table_list if x not in recruit_info_df['recruit_uid'].tolist()]
            print(uid_table_list)
            print(recruit_info_df['recruit_uid'].tolist())
            print(sub_set)
            #print(sub_set)
            #filtered_recruit_info_df = recruit_info_df[recruit_info_df['recruit_uid'].isin([sub_set])]
            for sub in sub_set:
                cur.execute(f'delete from recruit_info where recruit_uid = {sub}')
            print(f'{comp} 공고 삭제됨')
    conn.commit()
    conn.close()

In [54]:
recruit_uid_list

[149197,
 151828,
 156751,
 159350,
 171904,
 172569,
 173071,
 173529,
 174324,
 174495,
 174812,
 175906,
 39622,
 169809,
 173839]

In [30]:
recruit_info_df['recruit_uid']

0    173839
1    169809
2     39622
Name: recruit_uid, dtype: int64

In [ ]:
    ####
    #비교해주는 쿼리문 기존 테이블 데이터가 있을거란 말이지 #1. 테이블에recruit_uid와 크롤링한 recruit_info_df['recruit_uid'] 이 친구와
    # uid에는 있고 df에는 없으면 없어진거니 삭제 delete 
    if recruit_info_df['recruit_uid']
    for crawl_uid in recruit_info_df['recruit_uid']
    cur.execute('delete from recruit_info where recruit_uid NOT IN ('uid1', 'uid2', 'uid3', ...)')
    ####


In [ ]:
#생각해보니 굳이 테이블 생성할 필요없이 테이블에서 uid 다 select해오고 데이터프레임과 비교한뒤 걔네들을  

In [33]:
recruit_uid_list

[149197,
 151828,
 156751,
 159350,
 171904,
 172569,
 173071,
 173529,
 174324,
 174495,
 174812,
 175906,
 39622,
 169809,
 173839]

In [32]:
sub_set

[149197,
 151828,
 156751,
 159350,
 171904,
 172569,
 173071,
 173529,
 174324,
 174495,
 174812,
 175906,
 39622,
 169809,
 173839]

In [17]:
recruit_uid_list #채용공고 uid에 있는 uid 이제 df uid랑 비교해야겠지? 여기있는데 df는 없으면 

[149197,
 151828,
 156751,
 159350,
 171904,
 172569,
 173071,
 173529,
 174324,
 174495,
 174812,
 175906,
 39622,
 169809,
 173839]

In [14]:
recruit_info_df['recruit_uid']

0    169809
1    173839
2     39622
Name: recruit_uid, dtype: int64

In [13]:
for i in recruit_info_df['recruit_uid']:
    print(i)

169809
173839
39622


In [ ]:

cur.execute(f'select comp_uid from comp_info where comp_name = "{comp}"') 
comp_uid=cur.fetchall()[0][0]
print(comp_uid)
for index, row in recruit_info_df.iterrows():
    sql = 'insert into recruit_info '
    sql += '    (comp_uid, recruit_uid, recruit_url, recruit_position, recruit_thumb, create_date, modify_date) '
    sql += 'values ( '
    sql += f'   "{comp_uid}", "{row["recruit_uid"]}", "{row["recruit_url"]}", "{row["recruit_position"]}", "{row["recruit_thumb"]}" '
    sql += f'    , "{"00000000"}", "{"00000000"}" '
    sql += ') '
    cur.execute(sql)
    for i in cur:
        print(i)
